In [2]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras

2023-08-18 16:34:27.377381: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-18 16:34:28.071649: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
ROOT_URL = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

def read_data(filename):
    data = np.loadtxt(ROOT_URL + filename, delimiter="\t")
    y = data[:,0].astype(int)
    x = data[:,1:]
    
    return x,y


xtr, ytr = read_data("FordA_TRAIN.tsv")
xts, yts = read_data("FordA_TEST.tsv")

print(xtr.shape, ytr.shape, xts.shape, yts.shape)

classes = np.unique(np.concatenate((ytr, yts)))

# reshape into multivariate value
xtr = xtr.reshape((xtr.shape[0],xtr.shape[1],1))
xts = xts.reshape((xts.shape[0],xts.shape[1],1))
print(xtr.shape, xts.shape)

# shuffle
idx = np.random.permutation(xtr.shape[0])
xtr, ytr = xtr[idx], ytr[idx]

# labels tp positive ints
ytr[ytr == -1] = 0
yts[yts == -1] = 0

(3601, 500) (3601,) (1320, 500) (1320,)
(3601, 500, 1) (1320, 500, 1)


In [ ]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    x = keras.layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = keras.layers.Dropout(dropout)(x)
    x = keras.layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    x = keras.layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = keras.layers.Dropout(dropout)(x)
    x = keras.layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = keras.layers.LayerNormalization(epsilon=1e-6)(x)

    return x + res